In [1]:

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

import re

import pytz

from time import strptime
import calendar

from datetime import datetime, date
from pathlib import Path
import os
from icalendar import Calendar, Event,  vText, vCalAddress

In [2]:
def make_soup(url):
    
    request = requests.get(url)
    if request.status_code == 200:
        soup = BeautifulSoup(request.content, 'html.parser')
    else:
        print('failed: ' + request.status_code)
    
    return soup
    
# url = 'https://www.nuerburgring-langstrecken-serie.de/en/calendar-nurburgring-langstrecken-serie-2023/'

In [3]:
soup = make_soup('https://gotransam.com/events/2024-Trans-Am-National-Championship/cat/538')


In [4]:
# events = soup.find_all('div', {'class':'events-listbox'})
# test = events[3].find('div', {'class':'event-destination'}).text.split()
# # test2 = test.split('Tour  ')[1:]
# test

In [5]:
# events

In [6]:
import re

def TA_scrape_circuits():
    circuits = []
    events = soup.find_all('div', {'class':'events-listbox'})

    for event in events:
        event_text = event.find('div', {'class':'event-destination'}).text
        circuit = re.split('Tour\s*', event_text)[-1]  # Split on 'Tour' followed by any number of spaces
        circuits.append(circuit)

    return circuits

circuits = TA_scrape_circuits()
print(circuits)


['Sebring International Raceway\nSebring, FL\n', 'Road Atlanta \nBraselton, GA\n', 'NOLA Motorsports Park\nAvondale, LA\n', 'World Wide Technology Raceway  (TA2 Only)\nMadison, IL\n', 'Lime Rock Park\nLakeville, CT\n', 'Pittsburgh International Race Complex\nWampum, PA\n', 'Mid-Ohio Sports Car Course\nLexington, OH\n', 'Road America\nElkhart Lake, WI\n', 'Watkins Glen International\nWatkins Glen, NY\n', 'Canadian Tire Motorsport Park\nBowmanville, ON\n', 'VIRginia International Raceway\nAlton, VA\n', 'Circuit of The Americas\nAustin, TX\n']


In [7]:
def TA_create_titles():
    
    tit_prt = 'Trans-Am Race '
    rnd_cnt = 1
    titles = []
    
    events = soup.find_all('div', {'class':'events-listbox'})
    
    for i in events:
        tit = tit_prt + str(rnd_cnt)
        rnd_cnt += 1
        titles.append(tit) 

    return titles

titles = TA_create_titles()
titles

['Trans-Am Race 1',
 'Trans-Am Race 2',
 'Trans-Am Race 3',
 'Trans-Am Race 4',
 'Trans-Am Race 5',
 'Trans-Am Race 6',
 'Trans-Am Race 7',
 'Trans-Am Race 8',
 'Trans-Am Race 9',
 'Trans-Am Race 10',
 'Trans-Am Race 11',
 'Trans-Am Race 12']

In [8]:
events[0].find('div', {'class':'event-calendar'}).text.split()

NameError: name 'events' is not defined

In [ ]:
x = events[0].find('div', {'class':'event-calendar'}).text.split()
x

In [ ]:
scrp_mon = strptime(x[0],'%b').tm_mon #separates the month and changes to string digit '2'
scrp_mon = str(scrp_mon)
scrp_mon

In [10]:
events = soup.find('div', {'class':'event-calendar'}).text.split()
events

['Feb', '22-25']

In [ ]:
for event in events:
    day_ints = []
    scrp_daystr = []

    scrp_day = event[1].split('-') #separates the days, changes from '23-26' to ['23', '26']
    for i in scrp_day:
        day_ints.append(int(i)) # changes to integers [23, 26] needed for .arrange below
    scrp_days = np.arange(day_ints[0], day_ints[1]+1, 1).tolist() #creates a list with all days in the event length [23,24,25,26]
    for day in scrp_days:
        scrp_daystr.append(str(day)) # back to str
    scrp_daystr 
    
    dates = []
    year = str(datetime.now().year)
    for day in scrp_daystr:
        assy_date = year + ', ' + day + ', ' + scrp_mon
        dates.append(assy_date)

    dates
    
    wknd_dates = []
    for date in dates:
        date_obj = datetime.strptime(date, '%Y, %d, %m')
        year, month, day = date_obj.year, date_obj.month, date_obj.day
        if calendar.weekday(year, month, day) >= 5:
            wknd_dates.append(date)
    wknd_dates

In [ ]:
day_ints = []
scrp_daystr = []

scrp_day = event[1].split('-') #separates the days, changes from '23-26' to ['23', '26']
for i in scrp_day:
    day_ints.append(int(i)) # changes to integers [23, 26] needed for .arrange below
scrp_days = np.arange(day_ints[0], day_ints[1]+1, 1).tolist() #creates a list with all days in the event length [23,24,25,26]
for day in scrp_days:
    scrp_daystr.append(str(day)) # back to str
scrp_daystr

In [ ]:
dates = []
year = str(datetime.now().year)
for day in scrp_daystr:
    assy_date = year + ', ' + day + ', ' + scrp_mon
    dates.append(assy_date)

dates

In [ ]:
wknd_dates = []
for date in dates:
    date_obj = datetime.strptime(date, '%Y, %d, %m')
    year, month, day = date_obj.year, date_obj.month, date_obj.day
    if calendar.weekday(year, month, day) >= 5:
        wknd_dates.append(date)
wknd_dates

In [ ]:
wknd_dates

In [ ]:
def scrape_nls(soup):
    
    cln_days = []
    cln_mons = []
    year = 2023
    titles = []
    circuit = []
    
    tr = soup.find_all('tr')
    

## For each round item (tr) find date, trim day to race date only (ie 11-13 becomes just 13, fri-sat becomes just sat)
## store dates in clean_dates in format for iCalendar req's
    for i in range(len(soup.find_all('tr'))): 
        temp_date = soup.find_all('tr')[i].find_all('td')[0].text.strip().split('-')[1].split('.')

        temp_day = temp_date[0]
        if temp_day[0] == '0':
            temp_day = temp_day[1:]
        cln_days.append(temp_day)

        temp_mon = temp_date[1]
        if temp_mon[0] == '0':
            temp_mon = temp_mon[1:]
        cln_mons.append(temp_mon)

        cln_yr = temp_date[2]
        cln_yrs.append(cln_yr)

#     clean_dates = []
#     for yr, mon, day in zip(cln_yrs, cln_mons, cln_days):
#         temp_date_ = yr + ', ' + mon + ', ' + day
#         clean_dates.append(temp_date_)
##


## Pull the event names as description and store to description list
    descriptions = []
    for i in range(len(soup.find_all('tr'))):
        temp_desc = soup.find_all('tr')[i].find_all('td')[1].text.strip()
        descriptions.append(temp_desc)
##

## Make a simple title string for each index.
#  index 0 is preseason test days, mnaully added to list
   
    titles = []
    titles.append('NLS Test Days')
    
    count = 1
    for i in range(len(soup.find_all('tr')[1:9])):
        titles.append('NLS Race R0' + str(count))
        count += 1
#         titles.append(temp_tit)
    
    titles.append('NLS Alternative Date')

        
    return cln_yrs, cln_mons, cln_days, descriptions, titles

In [ ]:
cln_yrs, cln_mons, cln_days, descriptions, titles = scrape_nls(soup)

In [ ]:
cln_yrs

In [ ]:
cln_mons

In [ ]:
cln_days

In [ ]:
descriptions

In [ ]:
titles

In [ ]:
circuit = 'Nürburgring Gesamtrecke' '\n' 'Nürburg, Rhineland-Palatinate, Germany'# 
print(circuit)

In [ ]:
## Creating and Populating the Calendar

In [ ]:
import pytz
import icalendar
from datetime import datetime
from pathlib import Path
import os
from icalendar import Calendar, Event,  vText, vCalAddress

In [ ]:
def new_ical(series_name):
    
   
    series_name = Calendar()
    series_name.add('prodid', '-//My calendar product//example.com//')
    series_name.add('version', '2.0')
    
    return series_name



In [ ]:
NLS = None
NLS = new_ical(NLS)
NLS

In [ ]:


def make_events(cal):
    uid = 110
    circuit = 'Nürburgring Gesamtrecke' '\n' 'Nürburg, Rhineland-Palatinate, Germany'

    for title, description, year, month, day in zip(titles, descriptions, cln_yrs, cln_mons, cln_days):
        ievent = Event()
        ievent.add('summary', title) #Title of the event
        ievent.add('description', description)     #Description of event
        ievent.add('dtstart', datetime(int(year), int(month), int(day), 10, 0, 0, tzinfo=pytz.timezone('utc')))
        ievent.add('dtend', datetime(int(year), int(month), int(day), 14, 0, 0, tzinfo=pytz.timezone('utc')))
        ievent.add('dtstamp', datetime(2023, 3, 17, 0, 10, 0, tzinfo=pytz.timezone('utc')))
        ievent.add('location', vText(circuit))
        ievent.add('uid', uid)
        uid += 1
        cal.add_component(ievent)
        
    return cal
    

In [ ]:
NLS = make_events(NLS)
# NLS

In [ ]:
directory = str(Path(r'C:\Users\chris\Documents\Calendars')) + "/"
print("ics file will be generated at ", directory)
f = open(os.path.join(directory, 'NLS 2023.ics'), 'wb')
f.write(NLS.to_ical())
f.close()
